In [1]:
# spark libraries
import pyspark
from pyspark.sql.functions import max

# data wrangling
import pandas as pd
import numpy as np

## Acquire

#### Use spark! 

In [2]:
# create local spark enviroment
spark = pyspark.sql.SparkSession.builder.getOrCreate()

#### Read in the three quarters from 2019

In [4]:
df1 = spark.read.csv('../data/data_Q1_2019/*.csv', header=True)
df2 = spark.read.csv('../data/data_Q2_2019/*.csv', header=True)
df3 = spark.read.csv('../data/data_Q3_2019/*.csv', header=True)

In [5]:
for df in [df1,df2,df3]:
    print(len(df.columns))

129
129
129


#### Combine 2019 dataframes together

In [6]:
df_2019 = df1.union(df2).union(df3)

#### Read in the 4 quarters from 2018

In [7]:
df1 = spark.read.csv('../data/data_Q1_2018/*.csv', header=True)
df2 = spark.read.csv('../data/data_Q2_2018/*.csv', header=True)
df3 = spark.read.csv('../data/data_Q3_2018/*.csv', header=True)
df4 = spark.read.csv('../data/data_Q4_2018/*.csv', header=True)

In [8]:
for df in [df1,df2,df3,df4]:
    print(len(df.columns))

105
109
109
129


#### Since the columns don't match, find the missing columns

In [9]:
cols_df1 = df1.columns
cols_df4 = df4.columns
remove_traits_list = list(set(cols_df4).difference(cols_df1))

In [10]:
remove_traits_list = list(set(list4).difference(list1))

#### Remove missing columns

In [12]:
for trait in remove_traits_list:
    df2 = df2.drop(trait)
    df3 = df3.drop(trait)
    df4 = df4.drop(trait)

In [13]:
for df in [df1,df2,df3,df4]:
    print(len(df.columns))

105
105
105
105


#### Combine 2018 dataframes together

In [14]:
df_2018 = df1.union(df2).union(df3).union(df4)

#### Read in the four quarters from 2017

In [15]:
df1 = spark.read.csv('../data/data_Q1_2017/*.csv', header=True)
df2 = spark.read.csv('../data/data_Q2_2017/*.csv', header=True)
df3 = spark.read.csv('../data/data_Q3_2017/*.csv', header=True)
df4 = spark.read.csv('../data/data_Q4_2017/*.csv', header=True)

In [17]:
for df in [df1,df2,df3,df4]:
    print(len(df.columns))

95
95
95
95


#### Combine the 2017 dataframes together

In [18]:
df_2017 = df1.union(df2).union(df3).union(df4)

#### Read in the four quarters from 2016

In [19]:
df1 = spark.read.csv('../data/data_Q1_2016/*.csv', header=True)
df2 = spark.read.csv('../data/data_Q2_2016/*.csv', header=True)
df3 = spark.read.csv('../data/data_Q3_2016/*.csv', header=True)
df4 = spark.read.csv('../data/data_Q4_2016/*.csv', header=True)

In [20]:
for df in [df1,df2,df3,df4]:
    print(len(df.columns))

95
95
95
95


#### Combine the 2016 dataframes together

In [21]:
df_2016 = df1.union(df2).union(df3).union(df4)

#### Remove extra columms and combine years into one dataframe

In [27]:
for df in [df_2016, df_2017, df_2018, df_2019]:
    print(len(df.columns))

95
95
105
129


In [28]:
cols_2016 = df_2016.columns
cols_2019 = df_2019.columns
remove_traits_list = list(set(cols_2019).difference(cols_2016))

In [29]:
for trait in remove_traits_list:
    df_2018 = df_2018.drop(trait)
    df_2019 = df_2019.drop(trait)

In [30]:
for df in [df_2016, df_2017, df_2018, df_2019]:
    print(len(df.columns))

95
95
95
95


In [31]:
df = df_2019.union(df_2018).union(df_2017).union(df_2016)

#### Extract max values from top five SMART attributes and convert to pandas

In [22]:
def data_to_pandas(df):
    df = df.groupby('serial_number', 'model', 'capacity_bytes'
              ).agg(
                max('failure'), max('smart_9_raw'), max('smart_5_raw'), 
                max('smart_187_raw') , max('smart_197_raw'), max('smart_198_raw')
                )
    df = df.select("*").toPandas()
    return df

#### Save dfs as csv for easy access

In [23]:
# df = data_to_pandas(df_2019)
# df.to_csv(r'./hard_drives_2019.csv')

In [24]:
# df = data_to_pandas(df_2018)
# df.to_csv(r'./hard_drives_2018.csv')

In [25]:
# df = data_to_pandas(df_2017)
# df.to_csv(r'./hard_drives_2017.csv')

In [26]:
# df = data_to_pandas(df_2016)
# df.to_csv(r'./hard_drives_2016.csv')

In [32]:
df = data_to_pandas(df)
df.to_csv(r'./hard_drives_smart_5.csv')

> Will now code everying in pandas! 

## Prepare

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169073 entries, 0 to 169072
Data columns (total 9 columns):
serial_number         169072 non-null object
model                 169073 non-null object
capacity_bytes        169073 non-null object
max(failure)          169073 non-null object
max(smart_9_raw)      161975 non-null object
max(smart_5_raw)      161851 non-null object
max(smart_187_raw)    104189 non-null object
max(smart_197_raw)    161841 non-null object
max(smart_198_raw)    161841 non-null object
dtypes: object(9)
memory usage: 11.6+ MB


In [36]:
df.head()

,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_197_raw),max(smart_198_raw)
0,176FT026T,TOSHIBA MQ01ABF050M,500107862016,0,9965,0,None,0,0
1,17eddeea3c620010,DELLBOSS VD,480036847616,0,None,None,None,None,None
2,18K0A02ZF97G,TOSHIBA MG07ACA14TA,14000519643136,1,99,43,None,0,0
3,2AGHLPBY,HGST HUH721212ALN604,12000138625024,0,990,0,None,0,0
4,2AGM5WEY,HGST HUH721212ALN604,12000138625024,0,978,0,None,0,0
